In [19]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import pandas as pd
import random
from torch.utils.data import DataLoader
import numpy as np
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

# Set seed for reproducibility
def set_seed_(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed_(42)
set_seed(42)

In [20]:
# !pip install transformers==4.50.0

In [6]:
# !pip install datasets

In [7]:
# !pip install scikit-learn

In [ ]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Llama`


In [9]:
!nvidia-smi

Fri Dec 26 06:56:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:17:00.0 Off |                    0 |
| N/A   57C    P0             71W /  300W |   79517MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [10]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/workspace/prev/ChainofThought


In [11]:
from datasets import load_dataset

ds = load_dataset("thenlpresearcher/test_data_human_validated_eng_mar")

Generating test split: 100%|███████████████████████████████████████████████████| 54/54 [00:00<00:00, 3873.59 examples/s]


In [13]:
df_test = ds['test']
df_test

Dataset({
    features: ['sent_written', 'sent_meant', 'gt', 'punct_type'],
    num_rows: 54
})

In [15]:
len(df_test['sent_written'])

54

In [16]:
df_test['sent_written'][0]

'Chanting the choir raised the volume as the celebrant intoned the prayer.'

In [17]:
def inference(prompt, max_new_tokens=512, temperature=0.3):
    """
    Helper function for inference
    ----------
    Parameters
    ----------
    prompt: string
    prompt for the generative model
    
    max_new_tokens: int
    Maximum number of token to be genearted 
    
    temperature: float
    temperature to parameterize the softmax
    ----------
    Returns the generated text
    """
    
    
    model_inputs = stereotype_tokenizer([prompt], return_tensors="pt").to(device_stereo)

    # For removal of input string from the output string
    input_ids_cutoff = model_inputs.input_ids.size(dim=1)
    
    generated_ids = stereotype_model.generate(**model_inputs,
                                   max_new_tokens=max_new_tokens,
                                   top_p = 1,
                                   temperature = temperature,
                                   do_sample=True, 
                                   pad_token_id=stereotype_tokenizer.eos_token_id)
    
    completion = stereotype_tokenizer.decode(
    generated_ids[0][input_ids_cutoff:],
    skip_special_tokens=True)
    
    return completion

In [40]:
def get_responses_stereotype_with_zero_cot(sentences, max_new_tokens=128):
    # Chain-of-Thought prompt for multiple sentences
    def prompt(sentence):
        return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>

            You are an expert linguist and translator specializing in English-to-Marathi translation. You specialize in "Punctuation Restoration," the process of identifying how missing punctuation (commas, periods, exclamation marks) in English can change the semantic meaning of a sentence when translated into Marathi.

            Your goal is to resolve linguistic ambiguity caused by the absence of punctuation to ensure the translation captures the "Intended Meaning" rather than a literal, unpunctuated interpretation.

            <|eot_id|><|start_header_id|>user<|end_header_id|>

            Definitions:
            1. English (Written): The raw, unpunctuated, and ambiguous input sentence.
            2. English (Meant): The grammatically correct version of the sentence with restored punctuation that reflects the most logical intended human thought.
            3. Marathi (Translation): The final translation in Marathi script that matches the "English (Meant)" version.

            Steps for Analysis:
            Step 1: Analyze the "English (Written)" input for potential points of ambiguity (e.g., vocative cases, list items, or clause boundaries).
            Step 2: Determine where punctuation (specifically commas or periods) is missing.
            Step 3: Generate the "English (Meant)" version by restoring the punctuation to favor the most natural and logical human communication.
            Step 4: Translate the punctuated "English (Meant)" version into Marathi, ensuring that the meaning is preserved.

            Input English: {sentence}

            Please provide the response in the following format:
            Step 1 (Restoration): [The English (Meant) sentence]
            Make sure that the translation is in Devanagari Script.
            Step 2 (Translation): [The Marathi translation (Devanagari Script)]
            
            Reasoning: [Briefly explain why you chose this specific punctuation restoration]
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    
    messages = [[
                prompt(sentence)
    ] for sentence in sentences]
    
    responses = []
    for i in tqdm(range(len(messages))):
        responses.append(inference(messages[i][0]))
        
    return responses


def get_responses_stereotype_with_zero_zero(sentences, max_new_tokens=128):
    # Chain-of-Thought prompt for multiple sentences
    def prompt(sentence):
        return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an expert linguist and translator specializing in English-to-Marathi machine translation. Your expertise lies in resolving linguistic ambiguities caused by missing punctuation in English and providing accurate Marathi translations that capture the intended meaning.
            <|eot_id|><|start_header_id|>user<|end_header_id|>
            
            Task: Translate the following English sentence into Marathi.

            Contextual Instructions:
            1. The "Input English" is provided in a "Written" format, meaning it often lacks standard punctuation (commas, periods, exclamation marks).
            2. Lack of punctuation can drastically change the intended meaning (e.g., "Let's eat, Grandpa" vs "Let's eat Grandpa"). 
            3. Your goal is to identify the most logical "Intended Meaning" behind the unpunctuated text.
            4. Ensure that the Marathi translation reflects semantic intent based on the restored meaning.
            
            
            Input English: {sentence}
            
            Make sure that the translation is in Devanagari Script.
            Please provide the response in the following format:
            Marathi Translation (Devanagari Script): 
            
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    messages = [[
                prompt(sentence)
    ] for sentence in sentences]
    
    responses = []
    for i in tqdm(range(len(messages))):
        responses.append(inference(messages[i][0]))
        
    return responses


def get_responses_stereotype_with_few_cot(sentences, max_new_tokens=128):
    # Chain-of-Thought prompt for multiple sentences
    def prompt(sentence):
        return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>

            You are an expert linguist and translator specializing in English-to-Marathi translation. You specialize in "Punctuation Restoration," the process of identifying how missing punctuation (commas, periods, exclamation marks) in English can change the semantic meaning of a sentence when translated into Marathi.

            Your goal is to resolve linguistic ambiguity caused by the absence of punctuation to ensure the translation captures the "Intended Meaning" rather than a literal, unpunctuated interpretation.

            <|eot_id|><|start_header_id|>user<|end_header_id|>

            Definitions:
            1. English (Written): The raw, unpunctuated, and ambiguous input sentence.
            2. English (Meant): The grammatically correct version of the sentence with restored punctuation that reflects the most logical intended human thought.
            3. Marathi (Translation): The final translation in Marathi script that matches the "English (Meant)" version.

            Steps for Analysis:
            Step 1: Analyze the "English (Written)" input for potential points of ambiguity (e.g., vocative cases, list items, or clause boundaries).
            Step 2: Determine where punctuation (specifically commas or periods) is missing.
            Step 3: Generate the "English (Meant)" version by restoring the punctuation to favor the most natural and logical human communication.
            Step 4: Translate the punctuated "English (Meant)" version into Marathi, ensuring that the meaning is preserved.
            
            Some examples are as follows:
            1. Input English: These are the components required motor brushes, bearings, and wiring.
            English Meant: These are the components required: motor brushes, bearings, and wiring.
            Marathi Translation: आवश्यक असलेले घटक खालीलप्रमाणे आहेत: मोटार ब्रशेस, बेअरिंग्ज आणि वायरिंग.

            2. Input English: As the machine develops the forms we use to record data from past projects will be amended.
            English Meant: As the machine develops, the forms we use to record data from past projects will be amended.
            Marathi Translation: जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्पांतील डेटा रेकॉर्ड करण्यासाठी आम्ही वापरत असलेले फॉर्म्स सुधारित केले जातील.

            3. Input English: What we see, we believe what we hear, we register
            English Meant: What we see, we believe; what we hear, we register.
            Marathi Translation: जे पाहतो, त्यावर विश्वास ठेवतो; जे ऐकतो, त्याची नोंद घेतो.
            These are the components required motor brushes, bearings, and wiring.
            
            
            Input English: {sentence}
            

            Please provide the response in the following format:
            Step 1 (Restoration): [The English (Meant) sentence]
            Make sure that the translation is in Devanagari Script.
            Step 2 (Translation): [The Marathi translation (Devanagari Script)]
            
            Reasoning: [Briefly explain why you chose this specific punctuation restoration]
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    
    messages = [[
                prompt(sentence)
    ] for sentence in sentences]
    
    responses = []
    for i in tqdm(range(len(messages))):
        responses.append(inference(messages[i][0]))
        
    return responses


def get_responses_stereotype_with_few_zero(sentences, max_new_tokens=128):
    # Chain-of-Thought prompt for multiple sentences
    def prompt(sentence):
        return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an expert linguist and translator specializing in English-to-Marathi machine translation. Your expertise lies in resolving linguistic ambiguities caused by missing punctuation in English and providing accurate Marathi translations that capture the intended meaning.
            <|eot_id|><|start_header_id|>user<|end_header_id|>
            
            Task: Translate the following English sentence into Marathi.

            Contextual Instructions:
            1. The "Input English" is provided in a "Written" format, meaning it often lacks standard punctuation (commas, periods, exclamation marks).
            2. Lack of punctuation can drastically change the intended meaning (e.g., "Let's eat, Grandpa" vs "Let's eat Grandpa"). 
            3. Your goal is to identify the most logical "Intended Meaning" behind the unpunctuated text.
            4. Ensure that the Marathi translation reflects semantic intent based on the restored meaning.
            
            
            Some examples are as follows:
            1. Input English: These are the components required motor brushes, bearings, and wiring.
            Marathi Translation: आवश्यक असलेले घटक खालीलप्रमाणे आहेत: मोटार ब्रशेस, बेअरिंग्ज आणि वायरिंग.

            2. Input English: As the machine develops the forms we use to record data from past projects will be amended.
            Marathi Translation: जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्पांतील डेटा रेकॉर्ड करण्यासाठी आम्ही वापरत असलेले फॉर्म्स सुधारित केले जातील.

            3. Input English: What we see, we believe what we hear, we register
            Marathi Translation: जे पाहतो, त्यावर विश्वास ठेवतो; जे ऐकतो, त्याची नोंद घेतो.
            
            Input English: {sentence}
            
            Make sure that the translation is in Devanagari Script.
            Please provide the response in the following format:
            Marathi Translation (Devanagari Script): 
            
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    messages = [[
                prompt(sentence)
    ] for sentence in sentences]
    
    responses = []
    for i in tqdm(range(len(messages))):
        responses.append(inference(messages[i][0]))
        
    return responses


def get_metrics(true, predicted):
    conf_matrix = confusion_matrix(true, predicted)
    class_report = classification_report(true, predicted, digits=4)
    return conf_matrix, class_report

In [8]:
import re
import pandas as pd

def classify_stereotype_with_cot(sentences, responses):
    # Initialize lists for answers, reasons, sentences, responses, and answer classifications
    all_answers, all_reasons, all_sentences, all_responses, all_answer_classifications = [], [], [], [], []

    # Process each response in the batch
    for sentence, response in zip(sentences, responses):
        text = response
        text = text.replace('*','')
        all_responses.append(response)
        all_sentences.append(sentence)

        # Define regex patterns for extracting answer and reason
        answer_pattern = r"(?:\*\*Answer\*\*|Answer:)\s*(.*?)(?=\s*(?:Reason:|\Z))"  # non-greedy match between Answer and Reason (or end)
        reason_pattern = r"Reason:\s*(.*)"
        
        # Extract answer and reason using regex
        answer_match = re.search(answer_pattern, text)
        reason_match = re.search(reason_pattern, text)

        # Check if matches are found and extract
        if answer_match and reason_match:
            answer = answer_match.group(1).strip()
            reason = reason_match.group(1).strip()

            # Append to lists
            all_answers.append(answer)
            all_reasons.append(reason)

            # Classify the answer based on keywords
            answer_ = answer.lower()
            if "anti" in answer_:
                answer_classification = 0
            elif "stereotype" in answer_:
                answer_classification = 1
            elif "bias" in answer_:
                answer_classification = 4
            elif "without" in answer_:
                answer_classification = 2
            elif "neutral" in answer_:
                answer_classification = 3
            else:
                answer_classification = -1  # If no classification is found

            all_answer_classifications.append(answer_classification)
        else:
            print(f"Answer or Reason not found for sentence: {sentence}")
            # Handle missing answer/reason case by appending None or placeholders
            all_answers.append(None)
            all_reasons.append(None)
            all_answer_classifications.append(-1)  # Indicating a classification issue

    # Create a DataFrame from the collected data
    df = pd.DataFrame({
        'Sentence': all_sentences,
        'Response': all_responses,
        'Answer': all_answers,
        'Reason': all_reasons,
        'Answer_Classification': all_answer_classifications
    })

    # Save the DataFrame to a CSV (or other formats as needed)
#     df.to_csv("stereotype_classifications.csv", index=False)
    
    return df

In [21]:
# Load the first causal LM for stereotype detection
stereotype_model_name = 'meta-llama/Llama-3.1-8B-Instruct'#llama 3.1 8b

stereotype_model = AutoModelForCausalLM.from_pretrained(stereotype_model_name)
stereotype_tokenizer = AutoTokenizer.from_pretrained(stereotype_model_name)
stereotype_tokenizer.padding_side='left'


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.19s/it]


In [22]:
# Ensure pad_token_id is set to eos_token_id if not already set
if stereotype_tokenizer.pad_token_id is None:
    stereotype_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    stereotype_model.resize_token_embeddings(len(stereotype_tokenizer))


device_stereo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stereotype_model.to(device_stereo)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128257, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [23]:
print(device_stereo)

cuda


In [41]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_zero_cot(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [05:53<00:00,  6.55s/it]


In [42]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/llama_zero_cot.csv", index=False)

In [43]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_zero_zero(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [02:25<00:00,  2.69s/it]


In [44]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/llama_zero_zero.csv", index=False)

In [ ]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_few_cot(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

 17%|██████████████                                                                      | 9/54 [00:51<04:16,  5.69s/it]

In [ ]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/llama_few_cot.csv", index=False)

In [ ]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_few_zero(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

In [ ]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/llama_few_zero.csv", index=False)

In [ ]:
def get_responses_stereotype_with_original(sentences, max_new_tokens=128):
    # Chain-of-Thought prompt for multiple sentences
    def prompt(sentence):
        return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an expert linguist and translator specializing in English-to-Marathi machine translation. Your expertise lies in providing accurate Marathi translations that capture the intended meaning.
            <|eot_id|><|start_header_id|>user<|end_header_id|>
            
            Task: Translate the following English sentence into Marathi.
            
            Input English: {sentence}
            
            Make sure that the translation is in Devanagari Script.
            Please provide the response in the following format:
            Marathi Translation (Devanagari Script): 
            
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    messages = [[
                prompt(sentence)
    ] for sentence in sentences]
    
    responses = []
    for i in tqdm(range(len(messages))):
        responses.append(inference(messages[i][0]))
        
    return responses

In [ ]:
sentences = list(df_test['sent_meant'])
    
responses = get_responses_stereotype_with_original(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

In [ ]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/llama_original.csv", index=False)